In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# данные за 2019 год
agents2019 = pd.read_csv('counterparty-analysis/agents2019.csv')
agents2019.drop("Unnamed: 0",axis=1, inplace=True)

# из 2019 года берем только таргет, так как все остальные показатели есть в файле 2020 года
agents2019 = agents2019[['Наименование ДП', 'Макс. ПДЗ за 2019 год, дней',
       'Сред. ПДЗ за 2019 год, дней',
       'Кол-во просрочек свыше 5-ти дней за 2019 год, шт.',
       'Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.',
       'Кол-во раз ПДЗ за 2019 год, шт.']]

In [3]:
# данные за 2020 год
agents2020 = pd.read_csv('counterparty-analysis/agents2020.csv')

# факты не используем, будем использовать только сумму их весов (колонка Итого)
for col in agents2020.columns:
    if 'Факт' in col:
        agents2020.drop(col, axis=1, inplace=True)

# объединим таргет 2019 года (в качестве фичи) с данными за 2020 год
df20 = agents2019.merge(agents2020, how='outer', on='Наименование ДП')
# теперь таргет 19 года - фича (ПДЗ за последний год)
# в качестве фичей берем данные за последние 4 года - для 2020 года это период 2016-2019
df20.columns = [col.replace('2019', 'Y-1') for col in df20.columns]
df20.columns = [col.replace('2018', 'Y-2') for col in df20.columns]
df20.columns = [col.replace('2017', 'Y-3') for col in df20.columns]
df20.columns = [col.replace('2016', 'Y-4') for col in df20.columns]

# соберем таргет
df20['Факт просрочки'] = df20['Макс. ПДЗ за 2020 год, дней'].map(lambda x: int(x > 0))
df20['Просрочка более 30 дней'] = df20['Макс. ПДЗ за 2020 год, дней'].map(lambda x: int(x > 30))
df20['Просрочка 0-30'] = df20['Макс. ПДЗ за 2020 год, дней'].map(lambda x: int(x <= 30))

# таргет 20 больше не актуален - будет использован как фича для 21 года
df20.drop(['Макс. ПДЗ за 2020 год, дней',
       'Сред. ПДЗ за 2020 год, дней',
       'Кол-во просрочек свыше 5-ти дней за 2020 год, шт.',
       'Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.',
       'Кол-во раз ПДЗ за 2020 год, шт.'], axis=1, inplace=True)

In [4]:
# данные за 2021 год
df21 = pd.read_csv('counterparty-analysis/agents2021.csv')
# те контрагенты, у которых уже дефолт - не нужны
df21 = df21[df21['Статус']=='Действующая']

# таргет 2019 года уже использован в качестве фичи за 20 год - больше не нужен
df21.drop(["Unnamed: 0", 'Макс. ПДЗ за 2019 год, дней',
       'Сред. ПДЗ за 2019 год, дней',
       'Кол-во просрочек свыше 5-ти дней за 2019 год, шт.',
       'Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.',
       'Кол-во раз ПДЗ за 2019 год, шт.', 'Статус'], axis=1, inplace=True)

# факты не используем, будем использовать только сумму их весов (колонка Итого)
# так же не нужны данные за 2016 год, так как для 2021 года используем 2017-2020 года
for col in df21.columns:
    if 'Факт' in col or '2016' in col:
        df21.drop(col, axis=1, inplace=True)

# теперь таргет 20 года - фича (ПДЗ за последний год)
# в качестве фичей берем данные за последние 4 года - для 2021 года это период 2017-2020
df21.columns = [col.replace('2020', 'Y-1') for col in df21.columns]
df21.columns = [col.replace('2019', 'Y-2') for col in df21.columns]
df21.columns = [col.replace('2018', 'Y-3') for col in df21.columns]
df21.columns = [col.replace('2017', 'Y-4') for col in df21.columns]

# соберем таргет
df21['Факт просрочки'] = (df21['ПДЗ 1-30'] + df21['ПДЗ 31-90'] +
                          df21['ПДЗ 91-365'] + df21['ПДЗ более 365']).apply(lambda x: int(x >= 1))
df21['Просрочка более 30 дней'] = (df21['ПДЗ 31-90'] + df21['ПДЗ 91-365'] + 
                                   df21['ПДЗ более 365']).apply(lambda x: int(x >= 1))
df21['Просрочка 0-30'] = df21['ПДЗ 1-30']

# предыдущий таргет 21 больше не актуален
df21.drop(['ПДЗ 1-30', 'ПДЗ 31-90', 'ПДЗ 91-365', 'ПДЗ более 365'], axis=1, inplace=True)

In [5]:
# для 20 года нет отчетности по оценке потенциала контрагента - заполним их -1
df20['Оценка потенциала контрагента 1, руб.'] = -1
df20['Оценка потенциала контрагента 2, руб.'] = -1

In [6]:
# объединим в один датасет
df = pd.concat([df20, df21]).reset_index(drop=True)
df.drop('Наименование ДП', axis=1, inplace=True)
df

,"Макс. ПДЗ за Y-1 год, дней","Сред. ПДЗ за Y-1 год, дней","Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.","Кол-во раз ПДЗ за Y-1 год, шт.",Итого,"Y-4, Нематериальные активы, RUB","Y-3, Нематериальные активы, RUB","Y-2, Нематериальные активы, RUB","Y-1, Нематериальные активы, RUB",...,"Y-1, Прибыль (убыток) до налогообложения , RUB","Y-4, Прибыль (убыток) от продажи, RUB","Y-3, Прибыль (убыток) от продажи, RUB","Y-2, Прибыль (убыток) от продажи, RUB","Y-1, Прибыль (убыток) от продажи, RUB",Факт просрочки,Просрочка более 30 дней,Просрочка 0-30,"Оценка потенциала контрагента 1, руб.","Оценка потенциала контрагента 2, руб."
0,0,0.00,0,0.000000,0,10.0,2.895541e+06,6.245860e+06,9.050955e+06,9.885987e+06,...,3.603784e+09,3.280355e+09,6.200120e+09,8.716191e+08,3.658634e+09,1,0,1,-1.000000e+00,-1.000000e+00
1,0,0.00,0,0.000000,0,20.0,0.000000e+00,3.885350e+04,3.439490e+04,2.929936e+04,...,8.747516e+07,1.630064e+07,1.109172e+07,5.135732e+07,9.411019e+07,1,0,1,-1.000000e+00,-1.000000e+00
2,7,5.50,1,132825.299363,2,40.0,2.468153e+06,1.288025e+07,8.694904e+06,4.958599e+06,...,-6.456439e+08,4.148586e+08,1.611318e+08,-9.298981e+07,-1.207210e+08,1,0,1,-1.000000e+00,-1.000000e+00
3,0,0.00,0,0.000000,0,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.999298e+09,4.903117e+09,5.186553e+09,7.869977e+09,4.029232e+09,1,0,1,-1.000000e+00,-1.000000e+00
4,2,2.00,0,0.000000,2,20.0,5.503185e+05,5.210191e+05,4.490446e+05,3.987261e+05,...,4.960408e+10,2.338912e+10,3.727984e+10,5.307524e+10,5.622122e+10,1,0,1,-1.000000e+00,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,4,1.75,0,0.000000,4,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.008280e+06,0.000000e+00,0.000000e+00,4.192994e+06,6.487898e+06,1,0,1,5.095541e+06,5.095541e+06
849,0,0.00,0,0.000000,0,5.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,2.624204e+05
850,0,0.00,0,0.000000,0,65.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.284076e+06,0.000000e+00,0.000000e+00,4.573248e+05,8.932484e+06,1,0,1,3.184713e+06,3.184713e+06
851,0,0.00,0,0.000000,0,5.0,3.375796e+04,1.847134e+04,8.917197e+03,6.369427e+03,...,2.835159e+07,1.465019e+08,6.499554e+07,7.620446e+07,2.644204e+07,0,0,0,5.095541e+06,5.095541e+06


In [7]:
# заполним пустые значения -1, так как это означает что ни один из факторов не сработал
df['Итого'] = df['Итого'].fillna(-1)

In [8]:
df

,"Макс. ПДЗ за Y-1 год, дней","Сред. ПДЗ за Y-1 год, дней","Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.","Кол-во раз ПДЗ за Y-1 год, шт.",Итого,"Y-4, Нематериальные активы, RUB","Y-3, Нематериальные активы, RUB","Y-2, Нематериальные активы, RUB","Y-1, Нематериальные активы, RUB",...,"Y-1, Прибыль (убыток) до налогообложения , RUB","Y-4, Прибыль (убыток) от продажи, RUB","Y-3, Прибыль (убыток) от продажи, RUB","Y-2, Прибыль (убыток) от продажи, RUB","Y-1, Прибыль (убыток) от продажи, RUB",Факт просрочки,Просрочка более 30 дней,Просрочка 0-30,"Оценка потенциала контрагента 1, руб.","Оценка потенциала контрагента 2, руб."
0,0,0.00,0,0.000000,0,10.0,2.895541e+06,6.245860e+06,9.050955e+06,9.885987e+06,...,3.603784e+09,3.280355e+09,6.200120e+09,8.716191e+08,3.658634e+09,1,0,1,-1.000000e+00,-1.000000e+00
1,0,0.00,0,0.000000,0,20.0,0.000000e+00,3.885350e+04,3.439490e+04,2.929936e+04,...,8.747516e+07,1.630064e+07,1.109172e+07,5.135732e+07,9.411019e+07,1,0,1,-1.000000e+00,-1.000000e+00
2,7,5.50,1,132825.299363,2,40.0,2.468153e+06,1.288025e+07,8.694904e+06,4.958599e+06,...,-6.456439e+08,4.148586e+08,1.611318e+08,-9.298981e+07,-1.207210e+08,1,0,1,-1.000000e+00,-1.000000e+00
3,0,0.00,0,0.000000,0,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.999298e+09,4.903117e+09,5.186553e+09,7.869977e+09,4.029232e+09,1,0,1,-1.000000e+00,-1.000000e+00
4,2,2.00,0,0.000000,2,20.0,5.503185e+05,5.210191e+05,4.490446e+05,3.987261e+05,...,4.960408e+10,2.338912e+10,3.727984e+10,5.307524e+10,5.622122e+10,1,0,1,-1.000000e+00,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,4,1.75,0,0.000000,4,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.008280e+06,0.000000e+00,0.000000e+00,4.192994e+06,6.487898e+06,1,0,1,5.095541e+06,5.095541e+06
849,0,0.00,0,0.000000,0,5.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,2.624204e+05
850,0,0.00,0,0.000000,0,65.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.284076e+06,0.000000e+00,0.000000e+00,4.573248e+05,8.932484e+06,1,0,1,3.184713e+06,3.184713e+06
851,0,0.00,0,0.000000,0,5.0,3.375796e+04,1.847134e+04,8.917197e+03,6.369427e+03,...,2.835159e+07,1.465019e+08,6.499554e+07,7.620446e+07,2.644204e+07,0,0,0,5.095541e+06,5.095541e+06


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 75 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Макс. ПДЗ за Y-1 год, дней                        853 non-null    int64  
 1   Сред. ПДЗ за Y-1 год, дней                        853 non-null    float64
 2   Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.  853 non-null    int64  
 3   Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.  853 non-null    float64
 4   Кол-во раз ПДЗ за Y-1 год, шт.                    853 non-null    int64  
 5   Итого                                             853 non-null    float64
 6   Y-4, Нематериальные активы, RUB                   853 non-null    float64
 7   Y-3, Нематериальные активы, RUB                   853 non-null    float64
 8   Y-2, Нематериальные активы, RUB                   853 non-null    float64
 9   Y-1, Нематериальные а

In [10]:
df.to_csv('data.csv', index=False)